# PaDiM on SageMaker--训练

## 说明
本章内容为调用SageMaker进行训练，数据来自S3，训练后的模型放到S3。

## 运行环境
Kernel 选择pytorch_latest_p37。  
本文在boto3 1.17.12和sagemaker 2.26.0下测试通过。

In [ ]:
import boto3,sagemaker
print(boto3.__version__)
print(sagemaker.__version__)

## 训练

In [ ]:
#修改为自己的路径
input_data = 's3://junzhong/data/mvtec/bottle/'
output_data = 's3://junzhong/result/mvtec/'

In [ ]:
import boto3
iam = boto3.client('iam')
roles = iam.list_roles(PathPrefix='/service-role')
role=""
for current_role in roles["Roles"]:
    if current_role["RoleName"].startswith("AmazonSageMaker-ExecutionRole-"):
        role=current_role["Arn"]
        break
#如果role为空表示有问题，需要先打开https://cn-northwest-1.console.amazonaws.cn/sagemaker/home?region=cn-northwest-1#/notebook-instances/create以创建IAM Role
print(role)

In [ ]:
from sagemaker.pytorch import PyTorch

instance_type='ml.m5.2xlarge'

estimator = PyTorch(entry_point='training.py',
                     source_dir='./source',
                     role=role,
                     output_path=output_data,
                     framework_version='1.6.0',
                     hyperparameters={'data_path':"/opt/ml/input/data/training/", 'save_path':'/opt/ml/model'}, 
                     py_version='py3',
                     instance_count=1,
                     instance_type=instance_type,
                     use_spot_instances=True,
                     max_wait=432000,
                    )

In [ ]:
result = estimator.fit(input_data)

In [ ]:
import os
os.makedirs("result", exist_ok=True)

In [ ]:
!aws s3 cp $estimator.model_data ./result

In [ ]:
%%sh
cd result
tar zxvf model.tar.gz

In [ ]:
!pwd

In [ ]:
!mv result/temp_wide_resnet50_2/train.pkl ../2-inference/source/